In [1]:
import psydac.core.interface as inter
import matplotlib.pyplot as plt

import numpy as np
import scipy.sparse as sparse

import derivatives as der
import projectors as proj
import evaluation as eva
import integrate as integr

In [2]:
# ... define boundary conditions in each direction (True: periodic, False: homogeneous Dirichlet)
bc = [False, False, False]
bc_x, bc_y, bc_z = bc

# ... length of computational domain in each direction
Lx = 1
Ly = 1
Lz = 1

# ... number of elements in each direction and corresponding element boundaries
Nel_x = 6
Nel_y = 6
Nel_z = 6

dx = Lx/Nel_x
dy = Ly/Nel_y
dz = Lz/Nel_z

el_b_x = np.linspace(0, Lx, Nel_x + 1)
el_b_y = np.linspace(0, Ly, Nel_y + 1)
el_b_z = np.linspace(0, Lz, Nel_z + 1)

# ... degree of spline basis (of the 0-forms) in each direction
p = [2, 2, 2]
px, py, pz = p

# ... number of basis functions
Nbase = [Nel_x + px, Nel_y + py, Nel_z + pz] 
Nbase_x, Nbase_y, Nbase_z = Nbase

# ... number of degrees of freedom
Nbase_x_0 = Nbase_x - bc_x*px - (1 - bc_x)*2
Nbase_y_0 = Nbase_y - bc_y*py - (1 - bc_y)*2
Nbase_z_0 = Nbase_z - bc_z*pz - (1 - bc_z)*2


# ... knot vectors
if bc_x == True:
    Tx = inter.make_periodic_knots(px, Nbase_x)
else:
    Tx = inter.make_open_knots(px, Nbase_x)
    
if bc_y == True:
    Ty = inter.make_periodic_knots(py, Nbase_y)
else:
    Ty = inter.make_open_knots(py, Nbase_y)
    
if bc_z == True:
    Tz = inter.make_periodic_knots(pz, Nbase_z)
else:
    Tz = inter.make_open_knots(pz, Nbase_z)
    
T = [Tx, Ty, Tz]

In [3]:
# ... test projectors PI_0 and PI_1
if bc == True:
    Psi = lambda x, y, z : np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z)
    
    grad_Psi_x = lambda x, y, z: 2*np.pi*np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z)
    grad_Psi_y = lambda x, y, z: 2*np.pi*np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z)
    grad_Psi_z = lambda x, y, z: 2*np.pi*np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z)
    
else:
    Psi = lambda x, y, z : x*(1 - x)*y*(1 - y)*z*(1 - z)

    grad_Psi_x = lambda x, y, z: (1 - 2*x)*y*(1 - y)*z*(1 - z)
    grad_Psi_y = lambda x, y, z: (1 - 2*y)*x*(1 - x)*z*(1 - z)
    grad_Psi_z = lambda x, y, z: (1 - 2*z)*x*(1 - x)*y*(1 - y)



x_test = np.random.rand(1)
y_test = np.random.rand(1)
z_test = np.random.rand(1)

xtest = [x_test, y_test, z_test]

print('Projector 0')
print(Psi(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V0(proj.PI_0(Psi, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc_x), xtest, p, Nbase, T, bc)[0])

print('----------------------------')
print('Projector 1')
print(grad_Psi_z(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V1_z(proj.PI_1_z(grad_Psi_z, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc_x), xtest, p, Nbase, T, bc)[0])

Projector 0
0.002019793533460147
0.002019793533460147
----------------------------
Projector 1
0.0029203495043280494
0.00292034950432805


In [19]:
# ... test projectors PI_1 and PI_2
if bc == True:
    Ex = lambda x, y, z : np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z)
    Ey = lambda x, y, z : np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z)
    Ez = lambda x, y, z : np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z)
    
    curl_Ex = lambda x, y, z : 2*np.pi*(np.cos(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z) + np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z))
    curl_Ey = lambda x, y, z : 2*np.pi*(np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.cos(2*np.pi*z) + np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z))
    curl_Ez = lambda x, y, z : 2*np.pi*(np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z) + np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z))
    
else:
    Ex = lambda x, y, z: x*y*(1 - y)*z*(1 - z)
    Ey = lambda x, y, z: y*x*(1 - x)*z*(1 - z)
    Ez = lambda x, y, z: z*x*(1 - x)*y*(1 - y)

    curl_Ex = lambda x, y, z: z*x*(1 - x)*(1 - 2*y) - y*x*(1 - x)*(1 - 2*z)
    curl_Ey = lambda x, y, z: x*y*(1 - y)*(1 - 2*z) - z*(1 - 2*x)*y*(1 - y)
    curl_Ez = lambda x, y, z: y*(1 - 2*x)*z*(1 - z) - x*(1 - 2*y)*z*(1 - z)

x_test = np.random.rand(1)
y_test = np.random.rand(1)
z_test = np.random.rand(1)

print('Projector 1')
print(Ex(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V1_x(proj.PI_1_x(Ex, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc), x_test, y_test, z_test, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)[0])


print('----------------------------')
print('Projector 2')
print(curl_Ex(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V2_x(proj.PI_2_x(curl_Ex, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc), x_test, y_test, z_test, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)[0])

Projector 1
-0.17425859833940358
-0.172716832051201
----------------------------
Projector 2
-4.631722400135971
-4.545218292372691


In [20]:
# ... test projectors PI_2 and PI_3
if bc == True:
    Bx = lambda x, y, z : np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z)
    By = lambda x, y, z : np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z)
    Bz = lambda x, y, z : np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z)
    
    div_B = lambda x, y, z : 2*np.pi*(np.cos(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z) + np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.cos(2*np.pi*z) + np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z))

else:
    Bx = lambda x, y, z : x*(1 - x)*y*z
    By = lambda x, y, z : y*(1 - y)*x*z
    Bz = lambda x, y, z : z*(1 - z)*x*y
    
    div_B = lambda x, y, z : y*z + x*z + x*y - 6*x*y*z
    
x_test = np.random.rand(1)
y_test = np.random.rand(1)
z_test = np.random.rand(1)

print('Projector 2')
print(Bx(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V2_x(proj.PI_2_x(Bx, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc), x_test, y_test, z_test, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)[0])

print('----------------------------')
print('Projector 3')
print(div_B(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V3(proj.PI_3(div_B, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc), x_test, y_test, z_test, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)[0])

Projector 2
0.047086271059853184
0.04603836632491352
----------------------------
Projector 3
-3.542227723821819
-3.596738478862892


In [21]:
# ... check commuting diagram property (V0 --> V1)
vec_0 = GRAD.dot(proj.PI_0(Psi, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc))

vec_1_x = proj.PI_1_x(grad_Psi_x, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)
vec_1_y = proj.PI_1_y(grad_Psi_y, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)
vec_1_z = proj.PI_1_z(grad_Psi_z, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)

vec_1 = np.append(vec_1_x, np.append(vec_1_y, vec_1_z))

print('max. error :', np.abs(vec_0 - vec_1).max())

max. error : 7.624399358530098e-07


In [22]:
# check commuting diagram property (V1 --> V2)
vec_1_x = proj.PI_1_x(Ex, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)
vec_1_y = proj.PI_1_y(Ey, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)
vec_1_z = proj.PI_1_z(Ez, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)

vec_1 = CURL.dot(np.append(vec_1_x, np.append(vec_1_y, vec_1_z)))

vec_2_x = proj.PI_2_x(curl_Ex, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)
vec_2_y = proj.PI_2_y(curl_Ey, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)
vec_2_z = proj.PI_2_z(curl_Ez, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)

vec_2 = np.append(vec_2_x, np.append(vec_2_y, vec_2_z))

print('max. error :', np.abs(vec_1 - vec_2).max())

max. error : 1.6179488918077745e-07


In [23]:
# check commuting diagram property (V2 --> V3)
vec_2_x = proj.PI_2_x(Bx, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)
vec_2_y = proj.PI_2_y(By, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)
vec_2_z = proj.PI_2_z(Bz, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)

vec_2 = DIV.dot(np.append(vec_2_x, np.append(vec_2_y, vec_2_z)))

vec_3 = proj.PI_3(div_B, Nbase_x, Nbase_y, Nbase_z, px, py, pz, el_b_x, el_b_y, el_b_z, bc)

print('max. error :', np.abs(vec_2 - vec_3).max())

max. error : 3.3450846766147624e-08
